In [1]:
from fabric.api import *
import os, time, boto
import ConfigParser
import boto.ec2
CONFIG_FILE = "ep.cfg"

ep_ubuntu_14_04 = "i-9bbddd22"

config = ConfigParser.RawConfigParser()

In [2]:
config.read(CONFIG_FILE)

['ep.cfg']

In [3]:
MY_AMI = config.get('ec2', 'AMI')
SECURITY_GROUP = config.get('ec2', 'SECURITY_GROUP')
KEY_PATH = config.get('ec2', 'KEY_PATH')
INSTANCE_TYPE = config.get('ec2', 'INSTANCE_TYPE')

os.environ["AWS_ACCESS_KEY_ID"] = config.get('ec2', 'AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = config.get('ec2', 'AWS_SECRET_ACCESS_KEY')

host = None
try:
    host = config.get('ec2', 'HOST')
except:
    pass

In [60]:
MY_AMI

'ami-47a23a30'

In [61]:
if host is not None and host != '':
    env.hosts = [host, ]
    env.user = config.get('ec2', 'USER')
    env.key_filename = KEY_PATH
    print "Instance already created, using it: %s" % host

In [5]:
boto.Version

'2.38.0'

In [7]:
conn = boto.ec2.connect_to_region('eu-west-1')
image = conn.get_image(MY_AMI)
security_groups = conn.get_all_security_groups()

# check ssh access from own IP is allowed
try:
    [ep_group] = [x for x in security_groups if x.name == SECURITY_GROUP]
except ValueError:
    pass
try:
    [own_cidr_ip_grant] = [rule for rule in ep_group.rules if rule.grants[0].cidr_ip == my_ip+'/32']
except ValueError:
    raise ValueError('no rule for TCP/22 with own IP found in security group: %s' % SECURITY_GROUP)

NameError: name 'my_ip' is not defined

In [65]:
try:
    [own_cidr_ip_grant] = [rule for rule in ep_group.rules if rule.grants[0].cidr_ip == my_ip+'/32']
except ValueError:
    raise ValueError('no rule for TCP/22 with own IP found in security group: %s' % SECURITY_GROUP)

SecurityGroup:default

In [63]:
SECURITY_GROUP

'default'

In [64]:
security_groups

[SecurityGroup:rds-launch-wizard, SecurityGroup:default]

In [50]:
for group in security_groups:
    if group.name == 'default':
    #     print group.rules
        for rule in group.rules:
#             print rule
            if rule.grants[0].cidr_ip == my_ip+'/32':
                print "found rule for my IP"
                print rule.ip_protocol, rule.from_port, rule.to_port, rule.grants

found rule for my IP
tcp 22 22 [62.30.221.21/32]


In [42]:
security_groups[1].authorize('tcp', 22, 22, my_ip+'/32')

True

In [37]:
from urllib import urlopen
import re
def getPublicIp():
    data = str(urlopen('http://checkip.dyndns.com/').read())
    # data = '<html><head><title>Current IP Check</title></head><body>Current IP Address: 65.96.168.198</body></html>\r\n'

    return re.compile(r'Address: (\d+\.\d+\.\d+\.\d+)').search(data).group(1)

In [41]:
my_ip = getPublicIp()

In [67]:
env.own_ip  = getPublicIp()

In [73]:
"""
Check the instance security rule allows connection via SSL/22 from my IP
If it does not, add a new rule
:return:
"""

config = ConfigParser.RawConfigParser()
config.read(CONFIG_FILE)

MY_AMI = config.get('ec2', 'AMI')
SECURITY_GROUP = config.get('ec2', 'SECURITY_GROUP')
KEY_PATH = config.get('ec2', 'KEY_PATH')
INSTANCE_TYPE = config.get('ec2', 'INSTANCE_TYPE')
REGION = config.get('ec2', 'REGION')

os.environ["AWS_ACCESS_KEY_ID"] = config.get('ec2', 'AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = config.get('ec2', 'AWS_SECRET_ACCESS_KEY')

conn = boto.ec2.connect_to_region(REGION)
image = conn.get_image(MY_AMI)
security_groups = conn.get_all_security_groups()

if 'own_ip' not in env:
    env.own_ip = get_own_ip()

# check ssh access from own IP is allowed
try:
    [ep_group] = [x for x in security_groups if x.name == SECURITY_GROUP]
except ValueError:
    pass

In [74]:
ep_group

SecurityGroup:default

In [79]:
try:
    [own_cidr_ip_grant] = [rule for rule in ep_group.rules if filter(lambda grant: grant.cidr_ip == env.own_ip + '/32', rule.grants)]
except ValueError:
    print(
        'no rule for TCP/22 with own IP %(own_ip)s found in security group: %(sgroup)s' % {'own_ip': env.own_ip,
                                                                                           'sgroup': SECURITY_GROUP})
    ep_group.authorize('tcp', 22, 22, env.own_ip + '/32')

In [78]:
for rule in ep_group.rules:
    for grant in rule.grants:
        if grant.cidr_ip == my_ip+'/32':        
            print "found rule for my IP"
            print rule.ip_protocol, rule.from_port, rule.to_port, rule.grants

found rule for my IP
tcp 22 22 [137.222.114.241/32, 62.30.221.21/32]


In [80]:
own_cidr_ip_grant

IPPermissions:tcp(22-22)

In [85]:
try:
    [geonode_key] = [x for x in conn.get_all_key_pairs() if x.name == 'ep-host']
except ValueError:
    # this probably means the key is not defined
    # get the first one in the belt for now:
    print "GeoNode file not found in the server"
    geonode_key = conn.get_all_key_pairs()[0]

In [84]:
for x in conn.get_all_key_pairs():
    print x.name

ep-host


In [86]:
geonode_key.name

u'ep-host'

In [87]:
reservation = image.run(security_groups=[ep_group, ], key_name=geonode_key.name, instance_type=INSTANCE_TYPE)
instance = reservation.instances[0]

print "Firing up instance"

# Give it 10 minutes to appear online
for i in range(120):
    time.sleep(5)
    instance.update()
    print instance.state
    if instance.state == "running":
        break

if instance.state == "running":
    dns = instance.dns_name
    print "Instance up and running at %s" % dns

config.set('ec2', 'HOST', dns)
config.set('ec2', 'INSTANCE', instance.id)
env.hosts = [dns, ]
env.user = config.get('ec2', 'USER')
env.key_filename = KEY_PATH
with open(CONFIG_FILE, 'wb') as configfile:
    config.write(configfile)

print "ssh -i %s ubuntu@%s" % (KEY_PATH, dns)
print "Terminate the instance via the web interface %s" % instance

Firing up instance
pending
pending
pending
pending
pending
pending
pending
pending
pending
running
Instance up and running at ec2-52-30-102-56.eu-west-1.compute.amazonaws.com
ssh -i ~/.ssh/ep-host.pem ubuntu@ec2-52-30-102-56.eu-west-1.compute.amazonaws.com
Terminate the instance via the web interface Instance:i-bdc5a604


In [94]:
from pprint import pprint

In [96]:
reservations  = conn.get_all_reservations()
for reservation in reservations:    
    for i in reservation.instances:
        pprint(i.__dict__)

{'_in_monitoring_element': False,
 '_placement': eu-west-1a,
 '_previous_state': None,
 '_state': stopped(80),
 'ami_launch_index': u'0',
 'architecture': u'x86_64',
 'block_device_mapping': {u'/dev/sda1': <boto.ec2.blockdevicemapping.BlockDeviceType object at 0x103826f50>},
 'client_token': u'OUZTi1444991252371',
 'connection': EC2Connection:ec2.eu-west-1.amazonaws.com,
 'dns_name': '',
 'ebs_optimized': False,
 'eventsSet': None,
 'group_name': None,
 'groups': [<boto.ec2.group.Group object at 0x1038837d0>],
 'hypervisor': u'xen',
 'id': u'i-9bbddd22',
 'image_id': u'ami-47a23a30',
 'instance_profile': None,
 'instance_type': u't2.micro',
 'interfaces': [NetworkInterface:eni-ea5a50a3],
 'ip_address': None,
 'item': u'\n                ',
 'kernel': None,
 'key_name': u'ep-host',
 'launch_time': u'2015-10-16T15:40:59.000Z',
 'monitored': False,
 'monitoring': u'\n                    ',
 'monitoring_state': u'disabled',
 'persistent': False,
 'platform': None,
 'private_dns_name': u'ip

In [99]:
reservations = conn.get_all_instances(filters={"tag:Name" : "ep"})
instances = [i for r in reservations for i in r.instances]
print instances
instance = instances[0]

[Instance:i-9bbddd22]


In [98]:
instance.start()
print "Firing up instance"

Firing up instance
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped
stopped


KeyboardInterrupt: 

In [100]:
# Give it 10 minutes to appear online
for i in range(120):
    time.sleep(5)
    instance.update()
    print instance.state
    if instance.state == "running":
        break

if instance.state == "running":
    dns = instance.dns_name
    print "Instance up and running at %s" % dns

running
Instance up and running at ec2-52-16-241-174.eu-west-1.compute.amazonaws.com


In [104]:
instance.state

u'running'

In [103]:
instance.start()

In [12]:
access_key = config.get('ec2', 'AWS_ACCESS_KEY_ID')
secret_key = config.get('ec2', 'AWS_SECRET_ACCESS_KEY')

s3 = boto.connect_s3(access_key, secret_key)
bucket = s3.lookup('ep-static')
key = bucket.new_key('testkey')
key.set_contents_from_string('This is a test')
key.exists()
key.delete()

<Key: ep-static,testkey>

In [13]:
from boto.s3.connection import S3Connection
conn = S3Connection(access_key, secret_key)
rs = conn.get_all_buckets()
for b in rs:
    print b

<Bucket: ep-static>
